# Advanced Topics in Machine Learning - Natural Language Processing -- Group Assignment
> Students
> - Patrik Rác
> - Méline Trochon
> - Valentina Moretti
> - Mattia Colbertaldo

---



In [ ]:
# Imports
import os
import torch
import pandas

## 1. Investigate Dataset


We will investigate the dataset ***SQuAD2.0 (Stanford Question Answering Dataset)*** avaliable [here](https://rajpurkar.github.io/SQuAD-explorer/).

In [ ]:
# Set the directory to work in
WORKING_DIR = "./squad"
DATA_DIR = "./data"

In [ ]:
# Download the dataset
print("Downloading the DEV dataset of SQuAD2.0")
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2023-12-31 12:03:03--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json.3’

dev-v2.0.json.3     100%[===================>]   4.17M  --.-KB/s    in 0.1s    

2023-12-31 12:03:03 (32.6 MB/s) - ‘dev-v2.0.json.3’ saved [4370528/4370528]



In [ ]:
dev_dataframe = pandas.read_json("/content/dev-v2.0.json")
print(f"Size of the dataset: {dev_dataframe.size} (e.g. Categories of questions)")
dev_dataframe.head()

Size of the dataset: 70 (e.g. Categories of questions)


,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


In [ ]:
def count_paragraphs(row):
    return len(row["data"]["paragraphs"])

print(f"Number of individual paragraphs: {sum(dev_dataframe.apply(count_paragraphs, axis=1))}")


def count_questions(row):
  return sum([len(p["qas"]) for p in row["data"]["paragraphs"]])

print(f"Number of individual questions: {sum(dev_dataframe.apply(count_questions, axis=1))}")

def count_impossible_questions(row):
  n_impossible = 0
  for p in row["data"]["paragraphs"]:
    for q in p["qas"]:
      if q["is_impossible"]:
        n_impossible += 1
  return n_impossible


print(f"Number of impossible questions: {sum(dev_dataframe.apply(count_impossible_questions, axis=1))}")

Number of individual paragraphs: 1204
Number of individual questions: 11873
Number of impossible questions: 5945


In [ ]:
print(type(dev_dataframe.iloc[0]["data"]))

import json
print(json.dumps(dev_dataframe.iloc[1]["data"], sort_keys=False, indent=4))

<class 'dict'>
{
    "title": "Computational_complexity_theory",
    "paragraphs": [
        {
            "qas": [
                {
                    "question": "What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?",
                    "id": "56e16182e3433e1400422e28",
                    "answers": [
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        },
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        },
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        }
                    ],
                    "is_impossible": false
                },
           

In [ ]:
# Example of a Paragraph and a question
import textwrap
print("Category of given paragraph: {}\n".format(dev_dataframe.iloc[0]["data"]["title"]))

example_datum = dev_dataframe.iloc[0]["data"]["paragraphs"][0]
print("Context\n {}\n".format(textwrap.fill(example_datum["context"], 50)))

print("Example Question:\n{}\n".format(example_datum["qas"][0]["question"]))

print("Example Answer:\n{}\n".format(example_datum["qas"][0]["answers"][0]["text"]))

Category of given paragraph: Normans

Context
 The Normans (Norman: Nourmands; French: Normands;
Latin: Normanni) were the people who in the 10th
and 11th centuries gave their name to Normandy, a
region in France. They were descended from Norse
("Norman" comes from "Norseman") raiders and
pirates from Denmark, Iceland and Norway who,
under their leader Rollo, agreed to swear fealty
to King Charles III of West Francia. Through
generations of assimilation and mixing with the
native Frankish and Roman-Gaulish populations,
their descendants would gradually merge with the
Carolingian-based cultures of West Francia. The
distinct cultural and ethnic identity of the
Normans emerged initially in the first half of the
10th century, and it continued to evolve over the
succeeding centuries.

Example Question:
In what country is Normandy located?

Example Answer:
France



## TODO Section
---

- Investigate the data further (Possibly some plots etc.)
- Train Word2Vec on the corpus of paragraphs.
- (Potentially index the paragraphs) -> Find appropriate Datastructure potentially

=> Prepare Data for Training

---
- Investigate the application of `DistilBert`
  - Fine Tuning
  - One-Shot learning
  - (Two-Shot learning)
- Come up with alternatives (Custom Transformer etc.)
---


## Finetuning DistilBERT

In [ ]:
# Install the required Huggingface libs
! pip install datasets transformers accelerate -U

In [ ]:
# Import the transformers package (We'll be taking a pretrained model from here)
import transformers
print(transformers.__version__)

4.36.2


In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from datasets import load_dataset

squad2_datasets = load_dataset("squad_v2") # TODO: Replace with loading custom dataset

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

NameError: ignored

### Actual Fine Tuning of the Model

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

NameError: ignored

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("test-squad-trained")

In [ ]:
# TODO: Evaluation. The evaluation of the model should be rather genereal and work with all model training strategies.

# One shot learning

### QA data preparation

In this section we will be using the [WikiQA](https://aclanthology.org/D15-1237/) data set.
It's a data set for open domain generative QA.

It's avaialble via the HuggingFace data set package, let's install it

In [ ]:
!pip -q install datasets

In [ ]:
!pip -q install transformers==4.22.2
!pip -q install -U sentence-transformers

In [ ]:
!pip install --upgrade accelerate

Let's download the validation split of the dev_dataframe data set

In [ ]:
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import datasets

In [ ]:
# Print the first example from dev_dataframe
dev_dataframe.head()

,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


The data set contains questions, title of documents in Wikipedia containing the answers and answers.
Additionally the data set contains distractor answers for a given question.
We can distinguish the two types of responses from the `label` value.

Let's filter the data to retain only correct question-response pairs

In [ ]:
import pandas as pd

# Assuming your DataFrame is already loaded
# Replace 'your_file.json' with the actual file path if needed
dev_dataframe = pd.read_json("/content/dev-v2.0.json")

# Flatten the nested structure to work with individual questions
flat_dataframe = pd.json_normalize(dev_dataframe['data'], record_path=['paragraphs', 'qas'], meta=['title', 'paragraphs'])

# Filter rows where 'is_impossible' is False
dev_dataframe_possible = flat_dataframe[flat_dataframe['is_impossible'] == False]

# Display the first few rows of the filtered DataFrame
print(dev_dataframe_possible.head())


                                            question  \
0               In what country is Normandy located?   
1                 When were the Normans in Normandy?   
2      From which countries did the Norse originate?   
3                          Who was the Norse leader?   
4  What century did the Normans first gain their ...   

                         id  \
0  56ddde6b9a695914005b9628   
1  56ddde6b9a695914005b9629   
2  56ddde6b9a695914005b962a   
3  56ddde6b9a695914005b962b   
4  56ddde6b9a695914005b962c   

                                             answers  is_impossible  \
0  [{'text': 'France', 'answer_start': 159}, {'te...          False   
1  [{'text': '10th and 11th centuries', 'answer_s...          False   
2  [{'text': 'Denmark, Iceland and Norway', 'answ...          False   
3  [{'text': 'Rollo', 'answer_start': 308}, {'tex...          False   
4  [{'text': '10th century', 'answer_start': 671}...          False   

  plausible_answers    title  \
0               N

Ok now we have a lot of samples to try our system

In [ ]:
import json

row_index = 1

# Extract the relevant row
selected_row = dev_dataframe_possible.iloc[row_index]

# Convert the entire row to JSON and pretty print it
formatted_json = json.dumps(selected_row.to_dict(), sort_keys=False, indent=4)
print(formatted_json)

#TODO are not all false but the group in which there is at least one false

{
    "question": "When were the Normans in Normandy?",
    "id": "56ddde6b9a695914005b9629",
    "answers": [
        {
            "text": "10th and 11th centuries",
            "answer_start": 94
        },
        {
            "text": "in the 10th and 11th centuries",
            "answer_start": 87
        },
        {
            "text": "10th and 11th centuries",
            "answer_start": 94
        },
        {
            "text": "10th and 11th centuries",
            "answer_start": 94
        }
    ],
    "is_impossible": false,
    "plausible_answers": NaN,
    "title": "Normans",
    "paragraphs": [
        {
            "qas": [
                {
                    "question": "In what country is Normandy located?",
                    "id": "56ddde6b9a695914005b9628",
                    "answers": [
                        {
                            "text": "France",
                            "answer_start": 159
                        },
                       

### Knowlege retrieval

**We have**: the questions (and the target answers)

**We need**: knowledge source & the retrieval system.

We can re-use the simple Wikipedia and the two encoder models from last tutorial.

**DATA:**

In [ ]:
from transformers import BertConfig  # Replace BertConfig with the specific configuration class you need
from sentence_transformers import SentenceTransformer, CrossEncoder
! pip install --upgrade transformers

semb_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
xenc_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

  Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
  Using cached tokenizers-0.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.22.2
    Uninstalling transformers-4.22.2:
      Successfully uninstalled transformers-4.22.2


### Answering a question

We are still missing the core of the QA system, the answering model.
We are going to re-use a pre-trained model.

[FLAN T5](https://arxiv.org/abs/2210.11416) is a pre-trained encode-decoder model trained to be used on different tasks in zero-shot or few-shots leaning settings

In [ ]:
!pip -q install transformers sentencepiece accelerate

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Let's check quickly that it works:
zero shot learning: we give him a task he did not see during the training

In [ ]:
input_text = 'Translate the following sentence from Italian to English: "Amo la pizza"'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

output_ids = model.generate(input_ids, max_new_tokens=32)
output_text = tokenizer.decode(output_ids[0])
print(output_text)

RuntimeError: ignored

# Index all the paragraphs and all the questions

In [ ]:
import pandas as pd

# Replace 'path_al_tuo_file.json' with the actual file path
squad_data = dev_dataframe

# Extract questions and paragraphs and index them
questions = []
paragraphs = []

# Iterate over documents and paragraphs
for _, row in squad_data.iterrows():
    for paragraph in row['data']['paragraphs']:
        for qa in paragraph['qas']:
            question_dict = {
                'id': qa['id'],
                'title': row['data']['title'],
                'context': paragraph['context'],
                'question': qa['question'],
                'is_impossible': qa['is_impossible'],
                'answers': qa['answers']
            }
            questions.append(question_dict)

        # Append paragraphs to the separate list
        paragraph_dict = {
            'title': row['data']['title'],
            'context': paragraph['context']
        }
        paragraphs.append(paragraph_dict)

# Convert the lists to DataFrames
questions_df = pd.DataFrame(questions)
paragraphs_df = pd.DataFrame(paragraphs)

# Display the DataFrames
print("Questions:")
print(questions_df.head())

print("\nParagraphs:")
print(paragraphs_df.head())


Questions:
                         id    title  \
0  56ddde6b9a695914005b9628  Normans   
1  56ddde6b9a695914005b9629  Normans   
2  56ddde6b9a695914005b962a  Normans   
3  56ddde6b9a695914005b962b  Normans   
4  56ddde6b9a695914005b962c  Normans   

                                             context  \
0  The Normans (Norman: Nourmands; French: Norman...   
1  The Normans (Norman: Nourmands; French: Norman...   
2  The Normans (Norman: Nourmands; French: Norman...   
3  The Normans (Norman: Nourmands; French: Norman...   
4  The Normans (Norman: Nourmands; French: Norman...   

                                            question  is_impossible  \
0               In what country is Normandy located?          False   
1                 When were the Normans in Normandy?          False   
2      From which countries did the Norse originate?          False   
3                          Who was the Norse leader?          False   
4  What century did the Normans first gain their ...    

Now we can test our pipeline.
First select randomly a question from the data set

In [ ]:
import random

random.seed(10)

idx = random.choice(range(len(questions)))

sample = questions[idx]
print(sample)
question = sample['question']
target_answer = sample['answers']

print(f'Question {idx}: {question}')
print(f'Answer {idx}: {target_answer}')

idx = random.choice(range(len(paragraphs)))

sample = paragraphs[idx]
print(sample)
context = sample['context']


print(f'Context {idx}: {context}')

{'id': '572fff45947a6a140053cf27', 'title': 'Rhine', 'context': "Most of the Rhine's current course was not under the ice during the last Ice Age; although, its source must still have been a glacier. A tundra, with Ice Age flora and fauna, stretched across middle Europe, from Asia to the Atlantic Ocean. Such was the case during the Last Glacial Maximum, ca. 22,000–14,000 yr BP, when ice-sheets covered Scandinavia, the Baltics, Scotland and the Alps, but left the space between as open tundra. The loess or wind-blown dust over that tundra, settled in and around the Rhine Valley, contributing to its current agricultural usefulness.", 'question': 'What stretched across middle Europe in the last ice age?', 'is_impossible': False, 'answers': [{'text': 'tundra', 'answer_start': 137}, {'text': 'tundra', 'answer_start': 137}, {'text': 'A tundra', 'answer_start': 135}]}
Question 9361: What stretched across middle Europe in the last ice age?
Answer 9361: [{'text': 'tundra', 'answer_start': 137}, 

# Embed paragraphs and questions

Now let's embed the retrieved passages (We can checkpoint the embeddings to avoid repeating the computation each time)

In [ ]:
import os
import pickle

# Define hnswlib index path
embeddings_cache_path = './qa_embeddings_cache.pkl'

# Load cache if available
if os.path.exists(embeddings_cache_path):
    print('Loading embeddings cache')
    with open(embeddings_cache_path, 'rb') as f:
        corpus_embeddings = pickle.load(f)
# Else compute embeddings
else:
    print('Computing embeddings')
    corpus_embeddings = semb_model.encode(paragraphs, convert_to_tensor=True, show_progress_bar=True)
    # Save the index to a file for future loading
    print(f'Saving index to: \'{embeddings_cache_path}\'')
    with open(embeddings_cache_path, 'wb') as f:
        pickle.dump(corpus_embeddings, f)

Computing embeddings


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Saving index to: './qa_embeddings_cache.pkl'


Finally let's index the embeddings (and let's save the index hoping it works)

In [ ]:
!pip -q install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import hnswlib
import time
start = time.time()
# Create empthy index
index = hnswlib.Index(space='cosine', dim=corpus_embeddings.size(1))

# Define hnswlib index path
index_path = './qa_hnswlib_100.index'

# Load index if available
if os.path.exists(index_path):
    print('Loading index...')
    index.load_index(index_path)
# Else index data collection
else:
    # Initialise the index
    print('Start creating HNSWLIB index')
    index.init_index(max_elements=corpus_embeddings.size(0), ef_construction=100, M=64) # see https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md for parameter description
    # Compute the HNSWLIB index (it may take a while)
    index.add_items(corpus_embeddings.cpu(), list(range(len(corpus_embeddings))))
    # Save the index to a file for future loading
    print(f'Saving index to: {index_path}')
    index.save_index(index_path)

end = time.time()
print(f"Exectution time: {int((end - start) / 60)}:{int((end - start) % 60)} min:sec")

Start creating HNSWLIB index
Saving index to: ./qa_hnswlib_100.index
Exectution time: 0:0 min:sec


In [ ]:
import random

random.seed(90)

idx = random.choice(range(len(questions)))

sample = questions[idx]
question = sample['question']
target_answer = sample['answers']

print(f'Question {idx}: {question}?')

Question 3336: Who does not have the right to set up an enterprise without unjustified restrictions??


Embed the question

In [ ]:
question_embedding = semb_model.encode(question, convert_to_tensor=True)

Retrieve relevant documents keeping top $k$ matches

In [ ]:
corpus_ids, distances = index.knn_query(question_embedding.cpu(), k=64)
scores = 1 - distances

print("Cosine similarity model search results")
print(f"Query: \"{question}\"")
print("---------------------------------------")
for idx, score in zip(corpus_ids[0][:5], scores[0][:5]):
    print(f"Score: {score:.4f}\nDocument: \"{paragraphs[idx]}\"\n\n")

Cosine similarity model search results
Query: "Who does not have the right to set up an enterprise without unjustified restrictions?"
---------------------------------------
Score: 0.1705
Document: "{'title': 'European_Union_law', 'context': 'European Union law is a body of treaties and legislation, such as Regulations and Directives, which have direct effect or indirect effect on the laws of European Union member states. The three sources of European Union law are primary law, secondary law and supplementary law. The main sources of primary law are the Treaties establishing the European Union. Secondary sources include regulations and directives which are based on the Treaties. The legislature of the European Union is principally composed of the European Parliament and the Council of the European Union, which under the Treaties may establish secondary law to pursue the objective set out in the Treaties.'}"


Score: 0.1705
Document: "{'title': 'European_Union_law', 'context': 'European

Re-rank retrieved documents

In [ ]:
import numpy as np

# Initialize texts list
texts = [[] for _ in range(len(paragraphs))]

# Populate texts list
for paragraph_idx, paragraph in enumerate(paragraphs):
    for idx, context_key in enumerate(['context']):  # Adjust keys as needed
        texts[paragraph_idx].append(paragraph[context_key])


# Create model inputs
model_inputs = [(question, ' '.join(paragraph_texts)) for paragraph_texts in texts]

# Predict scores using the cross-encoder model
cross_scores = xenc_model.predict(model_inputs)

# Print results
print("Cross-encoder model re-ranking results")
print(f"Query: \"{question}\"")
print("---------------------------------------")
for idx in np.argsort(-cross_scores)[:5]:
    print(f"Score: {cross_scores[idx]:.4f}\nDocument: \"{paragraphs[idx]['context']}\"\n\n")


('Who does not have the right to set up an enterprise without unjustified restrictions?', 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.')
Cross-encoder model re-ranking results
Query: "Who does not have the right to set up an enterprise without unjustified restrictions?"
------------------------------

Use best match to answer (and compare to reference answer)

In [ ]:
passage_idx = np.argsort(-cross_scores)[0]

paragraphs = paragraphs[passage_idx]

input_text = f"Given the following paragrapha, answer the related question.\n\paragraphs:\n\n{paragraphs}\n\nQ: {question}?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
print(input_text, "\n")

'''
output_ids = model.generate(input_ids, max_new_tokens=32)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text, "\n")

print(f"A (target): {target_answer}")
'''

KeyError: ignored

How do we know if the passage was useful and the model haven't exploited weights memorisation?
Let's try to generate directly the response


In [ ]:
'''
# Create a input_text for the T5 model
input_text = f"Answer the following question.\n\nQ: {question}?"
print(input_text)
# tokenise the input_text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

# generate the output_ids
output_ids = model.generate(input_ids, max_new_tokens=32)
# decode the output_ids
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# print the output_text
print(f"\nA: {output_text}")
'''

Answer the following question.

Q: Who does not have the right to set up an enterprise without unjustified restrictions??


'\n# generate the output_ids\noutput_ids = model.generate(input_ids, max_new_tokens=32)\n# decode the output_ids\noutput_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)\n# print the output_text\nprint(f"\nA: {output_text}")\n'

TODO check the next cell

In [1]:
def qa_pipeline(
    question,
    similarity_model=semb_model,
    embeddings_index=index,
    re_ranking_model=xenc_model,
    generative_model=model,
    device=device
):
    if not question.endswith('?'):
        question = question + '?'
    # Embed question
    question_embedding = similarity_model.encode(question, convert_to_tensor=True)
    # Search documents similar to question in index
    corpus_ids, distances = embeddings_index.knn_query(question_embedding.cpu(), k=64)
    # Re-rank results
    xenc_model_inputs = [(question, paragraphs[idx]) for idx in corpus_ids[0]]
    cross_scores = re_ranking_model.predict(xenc_model_inputs)
    # Get best matching passage
    passage_idx = np.argsort(-cross_scores)[0]
    passage = paragraphs[passage_idx]
    # Encode input
    input_text = f"Given the following passage, answer the related question.\n\nPassage:\n\n{paragraph}\n\nQ: {question}"
    print('INPUT TEXT:', input_text, "\n")
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    # Generate output
    output_ids = generative_model.generate(input_ids, max_new_tokens=512)
    # Decode output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Return result
    return output_text

NameError: name 'semb_model' is not defined

In [ ]:
question = input("Ask a question >>> ")  # e.g., "How many fingers in a hand?", "What is the oldest newspaper in Chile?", ...
print()

print(qa_pipeline(question))